In [7]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [10]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-10-08 18:11:09--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.2’

book-crossings.zip. 100%[===================>]  24.88M  33.9MB/s    in 0.7s    

2025-10-08 18:11:11 (33.9 MB/s) - ‘book-crossings.zip.2’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [11]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [12]:
df_ratings.head()

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [13]:
df_books.head()

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [14]:
merged_df = pd.merge(df_ratings, df_books, on='isbn')
merged_df = merged_df[['isbn', 'title', 'user', 'rating']]
display(merged_df.head())

,isbn,title,user,rating
0,034545104X,Flesh Tones: A Novel,276725,0.0
1,0155061224,Rites of Passage,276726,5.0
2,0446520802,The Notebook,276727,0.0
3,052165615X,Help!: Level 1,276729,3.0
4,0521795028,The Amsterdam Connection : Level 4 (Cambridge ...,276729,6.0


In [15]:
book_counts = merged_df['isbn'].value_counts()
user_counts = merged_df['user'].value_counts()
valid_books = book_counts[book_counts >= 100].index
valid_users = user_counts[user_counts >= 200].index
merged_filtered = merged_df[
    (merged_df['isbn'].isin(valid_books)) &
    (merged_df['user'].isin(valid_users))
].reset_index(drop=True)

In [18]:
book_user_matrix = merged_filtered.pivot_table(
    index='title',
    columns='user',
    values='rating',
    fill_value=0
)
model_knn = NearestNeighbors(
    n_neighbors=6,
    metric='cosine',
    algorithm='brute'
)
model_knn.fit(book_user_matrix.values)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

In [21]:
def get_recommends(book_title):
    if book_title not in book_user_matrix.index:
        return [book_title, []]

    book_idx = book_user_matrix.index.get_loc(book_title)
    book_vector = book_user_matrix.iloc[book_idx].values.reshape(1, -1)

    distances, indices = model_knn.kneighbors(book_vector)

    rec_books = []
    for dist, idx in zip(distances.flatten()[1:], indices.flatten()[1:]):
        rec_title = book_user_matrix.index[idx]
        rec_books.append([rec_title, float(dist)])

    return [book_title, rec_books]

In [22]:
print(get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))"))

['The Queen of the Damned (Vampire Chronicles (Paperback))', [['The Vampire Lestat (Vampire Chronicles, Book II)', 0.4854866862297058], ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.47014564275741577], ['Interview with the Vampire', 0.26744866371154785], ['The Witching Hour (Lives of the Mayfair Witches)', 0.2637212872505188], ['Lasher: Lives of the Mayfair Witches (Lives of the Mayfair Witches)', 0.21665668487548828]]]


In [20]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Lovely Bones: A Novel', 0.284143328666687], ['I Know This Much Is True', 0.23535799980163574], ['The Surgeon', 0.2330949306488037], ['The Weight of Water', 0.23216557502746582], ['Tis: A Memoir', 0.2062903642654419]]]
You haven't passed yet. Keep trying!


# Task
Create a book recommendation model using NearestNeighbors based on user ratings. The model should recommend 5 books similar to a given book title. The process involves merging book and rating data, filtering users and books based on rating counts, creating a user-item matrix, and fitting a NearestNeighbors model. Finally, implement and test a function that takes a book title and returns the 5 nearest book recommendations with their distances.

## Merge data

### Subtask:
Merge the `filtered_df` with `df_ratings` to include user information.


**Reasoning**:
Merge the `filtered_df` and `df_ratings` DataFrames on the 'isbn' column to include user information and then display the head of the resulting DataFrame.



In [106]:
merged_filtered = pd.merge(filtered_df, df_ratings, on='isbn')
display(merged_filtered.head())

,isbn,title,user_x,rating_x,user_y,rating_y
0,0155061224,Rites of Passage,276726,5.0,276726,5.0
1,0155061224,Rites of Passage,276726,5.0,159181,0.0
2,052165615X,Help!: Level 1,276729,3.0,276729,3.0
3,0521795028,The Amsterdam Connection : Level 4 (Cambridge ...,276729,6.0,276729,6.0
4,038550120X,A Painted House,276744,7.0,276744,7.0
